In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import models,transforms,datasets
from torch.utils import data
import os
# script_dir = os.path.dirname(os.path.abspath(__file__))
script_dir = os.getcwd()

# 精简输出精度
torch.set_printoptions(2)

import math

def multibox_prior(data, sizes, ratios):
    """生成以每个像素为中心具有不同形状的锚框，生成的是归一化之后的，需要乘以h,w才是真实的锚框"""
    # in_height, in_width = data.shape[-2:]
    in_height = data.shape[-2]
    in_width = data.shape[-1]

    device, num_sizes, num_ratios = data.device, len(sizes), len(ratios)
    boxes_per_pixel = (num_sizes + num_ratios -1)
    size_tensor = torch.tensor(sizes, device=device)
    ratio_tensor = torch.tensor(ratios, device=device)

    # 为了将锚点移动到像素的中心，需要设置偏移量
    # 因为一个像素的高为1且宽为1，我们选择偏移中心0.5
    offset_h, offset_w = 0.5, 0.5
    steps_h = 1.0 / in_height # 在y轴上缩放步长
    steps_w = 1.0 / in_width # 在x轴上缩放步长

    # 生成锚框的所有中心点
    center_h = (torch.arange(in_height, device=device) + offset_h) * steps_h
    center_w = (torch.arange(in_width, device=device) + offset_w) * steps_w
    # 其中第一个输出张量填充第一个输入张量中的元素，各行元素相同；第二个输出张量填充第二个输入张量中的元素，各列元素相同
    shift_y, shift_x = torch.meshgrid(center_h, center_w, indexing='ij')
    shift_y, shift_x = shift_y.reshape(-1), shift_x.reshape(-1)

    # 生成“boxes_per_pixel”个高和宽，
    # 之后用于创建锚框的四角坐标(xmin, ymin, xmax, ymax)
    # numel(w) = boxes_per_pixel = (num_sizes + num_ratios -1) , 只考虑包含s1或r1的组合: (s1, r1), (s1, r2), ..., (s1, rm), (s2, r1), (s3, r1), ..., (sn, r1)
    # w = torch.cat((size_tensor * torch.sqrt(ratio_tensor[0]), sizes[0] * torch.sqrt(ratio_tensor[1:])))  # 处理矩形输入
    # h = torch.cat((size_tensor / torch.sqrt(ratio_tensor[0]), sizes[0] / torch.sqrt(ratio_tensor[1:])))
    
    w = torch.cat((size_tensor * torch.sqrt(ratio_tensor[0]), sizes[0] * torch.sqrt(ratio_tensor[1:]))) * math.sqrt(in_height / in_width)
    w0 = torch.cat((size_tensor * torch.sqrt(ratio_tensor[0]), sizes[0] * torch.sqrt(ratio_tensor[1:]))) * torch.sqrt(in_height / in_width)
    if w.tolist() == w0.tolist():
        print("w == w0")
    h = torch.cat((size_tensor / torch.sqrt(ratio_tensor[0]), sizes[0] / torch.sqrt(ratio_tensor[1:]))) * math.sqrt(in_width / in_height)
    # 除以2来获得半高和半宽，repeat_interleave()：在原有的tensor上，按每一个tensor复制。repeat()：根据原有的tensor复制n个，然后拼接在一起。
    anchor_manipulations = torch.stack((-w, -h, w, h)).T.repeat(in_height * in_width, 1) / 2

    # 每个中心点都将有"boxes_per_pixel"个锚框
    # 所以生成含所有锚框中心的网络，重复了"boxes_per_pixel"次
    # repeat_interleave()：在原有的tensor上，按每一个tensor复制。repeat()：根据原有的tensor复制n个，然后拼接在一起。
    out_grid = torch.stack([shift_x, shift_y, shift_x, shift_y], dim=1).repeat_interleave(boxes_per_pixel, dim=0)
    output = out_grid + anchor_manipulations
    # unsqueeze 函数用于在指定维度上给张量增加一个维度，从而改变张量的形状。在这里，unsqueeze(0) 表示在维度 0 上增加一个维度，即在张量的最外层添加一个维度。
    return output.unsqueeze(0)

/home/qlf/anaconda3/envs/d2l/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import matplotlib
matplotlib.use("Agg")  # 这一句一定要放在下面这句的前面
from matplotlib import pyplot as plt

# img = plt.imread('../img/catdog.jpg')
# h, w = img.shape[:2]

# print(h, w)
# X = torch.rand(size=(1, 3, 2, 3))
# Y = multibox_prior(X, sizes=[0.5], ratios=[1])
# print(Y)
# print(Y.shape)

In [3]:
boxes1 = torch.tensor(
    [
        [1, 1, 3, 2],
        [1.5, 1, 2.5, 2]
    ]
)
boxes2 = torch.tensor(
    [
        [2, 1.5, 3, 3]
    ]
)
inter_upperlefts = torch.maximum(boxes1[:, None, :2], boxes2[:, :2])
print(boxes1[:, None, :2] + boxes2[:, :2],inter_upperlefts, inter_upperlefts.shape)
inter_lowerrights = torch.minimum(boxes1[:, None, 2:], boxes2[:, 2:])
print(inter_lowerrights, inter_lowerrights.shape)

tensor([[[2.00, 1.50]],

        [[2.00, 1.50]]]) torch.Size([2, 1, 2])
tensor([[[3.00, 2.00]],

        [[2.50, 2.00]]]) torch.Size([2, 1, 2])
